In [1]:
import os
import sys
# add your module path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
# change your base path
os.chdir('../')
print(os.getcwd())

/home/yoyee/Documents/deepSfm


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
import numpy as np
import matplotlib.pyplot as plt

In [24]:
import torch
import torch.nn.functional as F
import torch.nn as nn

to_floatTensor = lambda x: torch.tensor(x).type(torch.FloatTensor)

from utils.utils import inv_warp_image_batch

H, W = 20, 30
img = torch.tensor(np.arange(H*W).reshape(H,W)).view(1, 1, H, W)
print("img: ", img)

img:  tensor([[[[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
            14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
            28,  29],
          [ 30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
            44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,
            58,  59],
          [ 60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,
            74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,
            88,  89],
          [ 90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
           104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
           118, 119],
          [120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133,
           134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
           148, 149],
          [150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162

In [25]:
device = 'cpu'
Batch, channel, H, W = img.shape
coor_cells = torch.stack(torch.meshgrid(torch.linspace(-1, 1, W), torch.linspace(-1, 1, H)), dim=2)
coor_cells = coor_cells.transpose(0, 1)
coor_cells = coor_cells.to(device)
coor_cells = coor_cells.contiguous()
print("coor_cells: ", coor_cells.shape)
print(coor_cells)
homography = torch.tensor(np.identity(3)).unsqueeze(0)

coor_cells:  torch.Size([20, 30, 2])
tensor([[[-1.0000, -1.0000],
         [-0.9310, -1.0000],
         [-0.8621, -1.0000],
         ...,
         [ 0.8621, -1.0000],
         [ 0.9310, -1.0000],
         [ 1.0000, -1.0000]],

        [[-1.0000, -0.8947],
         [-0.9310, -0.8947],
         [-0.8621, -0.8947],
         ...,
         [ 0.8621, -0.8947],
         [ 0.9310, -0.8947],
         [ 1.0000, -0.8947]],

        [[-1.0000, -0.7895],
         [-0.9310, -0.7895],
         [-0.8621, -0.7895],
         ...,
         [ 0.8621, -0.7895],
         [ 0.9310, -0.7895],
         [ 1.0000, -0.7895]],

        ...,

        [[-1.0000,  0.7895],
         [-0.9310,  0.7895],
         [-0.8621,  0.7895],
         ...,
         [ 0.8621,  0.7895],
         [ 0.9310,  0.7895],
         [ 1.0000,  0.7895]],

        [[-1.0000,  0.8947],
         [-0.9310,  0.8947],
         [-0.8621,  0.8947],
         ...,
         [ 0.8621,  0.8947],
         [ 0.9310,  0.8947],
         [ 1.0000,  0.8947]],


In [26]:
inv_warp_image_batch(to_floatTensor(img), to_floatTensor(homography), mode = 'bilinear')

/home/yoyee/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


tensor([[[[  0.0000,   1.0000,   2.0000,   3.0000,   4.0000,   5.0000,   6.0000,
             7.0000,   8.0000,   9.0000,  10.0000,  11.0000,  12.0000,  13.0000,
            14.0000,  15.0000,  16.0000,  17.0000,  18.0000,  19.0000,  20.0000,
            21.0000,  22.0000,  23.0000,  24.0000,  25.0000,  26.0000,  27.0000,
            28.0000,  29.0000],
          [ 30.0000,  31.0000,  32.0000,  33.0000,  34.0000,  35.0000,  36.0000,
            37.0000,  38.0000,  39.0000,  40.0000,  41.0000,  42.0000,  43.0000,
            44.0000,  45.0000,  46.0000,  47.0000,  48.0000,  49.0000,  50.0000,
            51.0000,  52.0000,  53.0000,  54.0000,  55.0000,  56.0000,  57.0000,
            58.0000,  59.0000],
          [ 60.0000,  61.0000,  62.0000,  63.0000,  64.0000,  65.0000,  66.0000,
            67.0000,  68.0000,  69.0000,  70.0000,  71.0000,  72.0000,  73.0000,
            74.0000,  75.0000,  76.0000,  77.0000,  78.0000,  79.0000,  80.0000,
            81.0000,  82.0000,  83.0000,  84.